In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util, models
from sklearn.feature_extraction import _stop_words as stop_words
from tqdm.notebook import tqdm
from rank_bm25 import BM25Okapi

import torch
import string
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
from collections import Counter

# the next cells related to each methods. run one of them only

In [ ]:
# for QR512
bi_enc_weights = '/content/drive/MyDrive/microservice/bienc-exp7/'
cr_enc_weights = '/content/drive/MyDrive/microservice/crenc-readme-exp2/'
data_folder = 'generated5'
top_k = 50
use_base = False

import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):

    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Reconstruct the text
    return ' '.join(tokens)
file_path = '/content/drive/MyDrive/microservice/20231004_data.xlsx'
df = pd.read_excel(file_path, index_col=0)

df['readme_short'] = df['readme_short'].astype(str)


df['answer'] = df['readme_short'].apply(lambda x: advanced_clean(x))



df.head(2)

In [ ]:
# for QD

bi_enc_weights = '/content/drive/MyDrive/microservice/bienc-exp7/'
cr_enc_weights = '/content/drive/MyDrive/microservice/crenc-docker-exp2/'
data_folder = 'generated5'
top_k = 50
use_base = False

import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):

    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)


    tokens = word_tokenize(text)


    tokens = [token.lower() for token in tokens]


    tokens = [token for token in tokens if token not in stop_words]


    tokens = [lemmatizer.lemmatize(token) for token in tokens]


    return ' '.join(tokens)
file_path = '/content/drive/MyDrive/microservice/20231004_data.xlsx'
df = pd.read_excel(file_path, index_col=0)

df['docker'] = df['docker'].astype(str)


df['answer'] = df['docker'].apply(lambda x: advanced_clean(x))



df.head(2)

In [ ]:
# for QRD
bi_enc_weights = '/content/drive/MyDrive/microservice/bienc-exp7/'
cr_enc_weights = '/content/drive/MyDrive/microservice/crenc-readme-exp4/'
data_folder = 'generated5'
top_k = 50
use_base = False

import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):

    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)


    tokens = word_tokenize(text)


    tokens = [token.lower() for token in tokens]


    tokens = [token for token in tokens if token not in stop_words]


    tokens = [lemmatizer.lemmatize(token) for token in tokens]


    return ' '.join(tokens)
file_path = '/content/drive/MyDrive/microservice/20231004_data.xlsx'
df = pd.read_excel(file_path, index_col=0)

df['readme_short'] = df['readme_short'].astype(str)
df['docker'] = df['docker'].astype(str)

df['answer'] = df.apply(lambda row: advanced_clean(' '.join(row['readme_short'].split()[:300] + row['docker'].split()[:200])), axis=1)



df.head(2)

In [ ]:
print(df['answer'].iloc[0])

In [ ]:
english_stopwords = set(stopwords.words('english'))

def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in english_stopwords:
      tokenized_doc.append(token)

  return tokenized_doc

In [ ]:
if use_base:
    word_embedding_model = models.Transformer('distilroberta-base', max_seq_length=350)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    cr_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')
else:

    bi_encoder = SentenceTransformer(bi_enc_weights)

    cr_encoder = CrossEncoder(cr_enc_weights)

In [ ]:
with open(f'/content/drive/MyDrive/microservice/test_passage.json', 'r') as f:
    val_passage = json.load(f)

with open(f'/content/drive/MyDrive/microservice/test_corpus.json', 'r') as f:
    val_corpus = json.load(f)

val_query_answer = {}
val_query_readme = {}
for idx, rel in val_passage.items():
    pos = rel[0]


    readme = df.loc[int(pos[0]), 'answer']
    questions = []
    for p in pos:
        question = df.loc[int(p), 'Question Title']
        questions.append(question)
    val_query_answer[idx] = questions
    val_query_readme[idx] = [readme]

In [ ]:
def shorten(text):
    tmp = text.split()[:512]
    return ' '.join(tmp)

val_text = list(val_corpus.values())

val_readme = []
for t in val_text:
    val_readme.append(df.loc[df['Question Title'] == t, 'answer'].values[0])


with open("/content/drive/MyDrive/microservice/embeddings_GPT.json", "r") as jsonfile:
    embeddings_dict = json.load(jsonfile)

val_emb_tensors = []


title_to_id = dict(zip(df['Question Title'], df.index))

# Convert the embeddings into tensors
for text in val_text:

    text_id = title_to_id[text]
    embedding = embeddings_dict.get(str(text_id))
    if embedding:
        val_emb_tensors.append(torch.tensor(embedding))


val_emb = torch.stack(val_emb_tensors)


In [ ]:
from tqdm import tqdm
tokenized_corpus = []
for idx, passage in tqdm(val_corpus.items()):
    tokenized_corpus.append(bm25_tokenizer(passage))

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def evaluate_bm25(query, answer, top_k=50):
    # Convert answer to set for faster lookup
    answer_set = set(answer)

    # Get bm25 scores for the query
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))

    # Check if we have enough scores
    if len(bm25_scores) < top_k:
        raise ValueError("Not enough BM25 scores for top_k.")

    # Retrieve the top-k indices based on bm25_scores
    top_n = np.argpartition(bm25_scores, -top_k)[-top_k:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    bm25_counter = -1
    bm25_map = 0
    tmp_hits = 0

    # Iterate over top-k hits
    for idx, hit in enumerate(bm25_hits):
        candidate = val_text[hit['corpus_id']]
        if candidate in answer_set:
            if bm25_counter == -1:
                bm25_counter = idx + 1

            tmp_hits += 1
            bm25_map += tmp_hits / (idx + 1)
            answer_set.remove(candidate)

    bm25_map /= len(answer)
    bm25_mrr = 1 / bm25_counter if bm25_counter != -1 else 0.0

    return bm25_map, bm25_mrr

def compute_cosine_similarity(query_embedding, corpus_embeddings):
    # Normalize embeddings
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding / torch.norm(query_embedding, keepdim=True)
        query_embedding = query_embedding.unsqueeze(0)  # Add an additional dimension
    else:
        query_embedding = query_embedding / torch.norm(query_embedding, dim=1, keepdim=True)

    corpus_embeddings = corpus_embeddings / torch.norm(corpus_embeddings, dim=1, keepdim=True)

    # Compute cosine similarity
    cosine_similarities = torch.mm(query_embedding, corpus_embeddings.transpose(0, 1))

    return cosine_similarities


def forward_pass_rerank(query, precomputed_embedding=None, val_embeddings=None, top_k=50):

    if precomputed_embedding is None:
        q_emb = bi_encoder.encode(query, convert_to_tensor=True)
    else:
        q_emb = precomputed_embedding

    # Ensure the query embedding is 2-dimensional
    if len(q_emb.shape) == 1:
        q_emb = q_emb.unsqueeze(0)

    if val_embeddings is None:
        raise ValueError("No embeddings provided for the validation set.")


    if val_embeddings.shape[1] != q_emb.shape[1]:
        val_embeddings = val_embeddings.transpose(0, 1)

    cosine_similarities = compute_cosine_similarity(q_emb, val_embeddings)


    top_indices = torch.topk(cosine_similarities, k=top_k+1, dim=1).indices[0].tolist()
    hits = [{'corpus_id': index, 'score': cosine_similarities[0, index].item()} for index in top_indices]


    cross_inputs = []
    to_remove = -1
    for hit in hits:
        readme = val_readme[hit['corpus_id']]
        text = val_text[hit['corpus_id']]
        if query == text:
            to_remove = hits.index(hit)
        cross_inputs.append([query, readme])
    cross_scores = cr_encoder.predict(cross_inputs)
    for idx in range(len(cross_scores)):
        hits[idx]['cross_score'] = cross_scores[idx]
    if to_remove != -1: del hits[to_remove]
    hits = hits[:top_k]

    return hits


def evaluate_bi_encoder(hits, answer, top_k=50):
    answer_set = set(answer)
    bi_enc_counter = -1
    bi_enc_map = 0
    tmp_hits = 0
    bi_enc_hit_list = [0] * top_k
    bi_enc_hit_recall_list = [0] * top_k
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)

    for idx, hit in enumerate(hits):
        candidate = val_text[hit['corpus_id']]
        if candidate in answer_set:
            if bi_enc_counter == -1:
                bi_enc_counter = idx + 1
            bi_enc_hit_list[idx] = 1
            bi_enc_hit_recall_list[idx] = 1
            tmp_hits += 1
            bi_enc_map += tmp_hits / (idx + 1)
            answer_set.remove(candidate)

    bi_enc_map /= len(answer)
    bi_enc_mrr = 1 / bi_enc_counter if bi_enc_counter != -1 else 0.0

    return bi_enc_map, bi_enc_mrr, bi_enc_hit_list, bi_enc_hit_recall_list

def evaluate_cr_encoder(hits, answer, top_k=50, mode='cross_score'):
    answer_set = set(answer)
    cr_enc_counter = -1
    cr_enc_map = 0
    tmp_hits = 0
    cr_enc_hit_list = [0] * top_k
    cr_enc_hit_recall_list = [0] * top_k
    hits = sorted(hits, key=lambda x: x[mode], reverse=True)

    for idx, hit in enumerate(hits):
        candidate = val_readme[hit['corpus_id']]
        if candidate in answer_set:
            if cr_enc_counter == -1:
                cr_enc_counter = idx + 1
            cr_enc_hit_list[idx] = 1
            cr_enc_hit_recall_list[idx] = 1
            tmp_hits += 1
            cr_enc_map += tmp_hits / (idx + 1)
            answer_set.remove(candidate)

    cr_enc_map /= len(answer)
    cr_enc_mrr = 1 / cr_enc_counter if cr_enc_counter != -1 else 0.0

    return cr_enc_map, cr_enc_mrr, cr_enc_hit_list, cr_enc_hit_recall_list

bm25_scores = {'mrr': 0, 'map': 0}
bi_enc_scores = {'mrr': 0, 'map': 0, 'precision': [0] * 4, 'recall': [0] * 4}
cr_enc_scores = {'mrr': 0, 'map': 0, 'precision': [0] * 4, 'recall': [0] * 4}

for (query_key, answers), (_, readme) in tqdm(zip(val_query_answer.items(), val_query_readme.items()), total=len(val_query_answer)):
    query= val_corpus[query_key]
    # Fetch precomputed embedding
    query_id = title_to_id[query]
    precomputed_query_embedding = torch.tensor(embeddings_dict.get(str(query_id)))
    hits = forward_pass_rerank(query, precomputed_embedding=precomputed_query_embedding, val_embeddings=val_emb, top_k=top_k)

    r = val_query_readme[query_key]
    #hits = forward_pass_rerank(query, top_k)


    bm25_map, bm25_mrr = evaluate_bm25(query, answers, top_k)
    b_map, b_mrr, b_hit, b_rec = evaluate_bi_encoder(hits, answers)
    c_map, c_mrr, c_hit, c_rec = evaluate_cr_encoder(hits, r)

    tmp_precision = [0] * 4
    tmp_recall = [0] * 4
    for idx, n in enumerate([1, 3, 5, 10]):
        tmp_precision[idx] = sum(b_hit[:n]) / n
        tmp_recall[idx] = sum(b_rec[:n]) / len(answers)

    bi_enc_scores['precision'] = [x + y for (x, y) in zip(bi_enc_scores['precision'], tmp_precision)]
    bi_enc_scores['recall'] = [x + y for (x, y) in zip(bi_enc_scores['recall'], tmp_recall)]

    tmp_precision = [0] * 4
    tmp_recall = [0] * 4
    for idx, n in enumerate([1, 3, 5, 10]):
        tmp_precision[idx] = sum(c_hit[:n]) / n
        tmp_recall[idx] = sum(c_rec[:n]) / len(answers)

    cr_enc_scores['precision'] = [x + y for (x, y) in zip(cr_enc_scores['precision'], tmp_precision)]
    cr_enc_scores['recall'] = [x + y for (x, y) in zip(cr_enc_scores['recall'], tmp_recall)]

    bm25_scores['map'] += bm25_map
    bm25_scores['mrr'] += bm25_mrr

    bi_enc_scores['map'] += b_map
    bi_enc_scores['mrr'] += b_mrr

    cr_enc_scores['map'] += c_map
    cr_enc_scores['mrr'] += c_mrr

bm25_scores['map'] /= len(val_query_answer)
bm25_scores['mrr'] /= len(val_query_answer)
bi_enc_scores['map'] /= len(val_query_answer)
bi_enc_scores['mrr'] /= len(val_query_answer)
bi_enc_scores['precision'] = [x/len(val_query_answer) for x in bi_enc_scores['precision']]
bi_enc_scores['recall'] = [x/len(val_query_answer) for x in bi_enc_scores['recall']]
cr_enc_scores['map'] /= len(val_query_answer)
cr_enc_scores['mrr'] /= len(val_query_answer)
cr_enc_scores['precision'] = [x/len(val_query_answer) for x in cr_enc_scores['precision']]
cr_enc_scores['recall'] = [x/len(val_query_answer) for x in cr_enc_scores['recall']]



In [ ]:
print(f'BM25:\n{json.dumps(bm25_scores, indent=2)}')
print(f'Bi-Encoder:\n{json.dumps(bi_enc_scores, indent=2)}')
print(f'Cross-Encoder:\n{json.dumps(cr_enc_scores, indent=2)}')